In [1]:
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import folium
import requests 
from geopy.geocoders import Nominatim
import re as regex

use request to connect to wikipedia page and use beautifulsoup to process html

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req=urllib.request.urlopen(url)
article=req.read().decode()
with open('localfile.html', 'w') as fo:
    fo.write(article)
article

article=open('localfile.html').read()
soup=BeautifulSoup(article,'html.parser')
tables=soup.find_all('table',class_='sortable')
tables

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighbourhood
 </th></tr>
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>
 <tr>
 <td>M2B

content to dataframe (at the time when I process wikipedia data, neighborhoods are already collapsed according to Borough)

In [3]:
results_table=[]
for tr in tables:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    results_table.append(row)
results_table=pd.DataFrame(np.array(results_table[0]).reshape((-1,3)))
results_table.columns=['PostalCode','Borough','Neighborhood']
results_table=results_table[results_table['Borough']!='Not assigned'].reset_index(drop=True)
results_table

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
results_table.shape


(103, 3)

download geographial information

In [5]:
!wget -q -O 'geo.csv' https://cocl.us/Geospatial_data

merge two dataset

In [6]:
geo=pd.DataFrame.from_csv('geo.csv')
results_table=results_table.merge(geo,left_on='PostalCode',right_index=True)
results_table


/Users/lesliezhao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


create a map for toronto

In [7]:
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(results_table['Latitude'], results_table['Longitude'], results_table['Borough'], results_table['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [9]:
toronto_data = results_table[results_table['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


four square information

In [10]:
CLIENT_ID = '50OG1L24VQHMKEDMSCTEKVCRVM3YZ2LLOOGNCNN4QVKN2SGY' # your Foursquare ID
CLIENT_SECRET = 'QMI5S45Y5GFI0MJT0BGORIW23LEFUNNMU1FE4UZJ5V4DOHGG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 50OG1L24VQHMKEDMSCTEKVCRVM3YZ2LLOOGNCNN4QVKN2SGY
CLIENT_SECRET:QMI5S45Y5GFI0MJT0BGORIW23LEFUNNMU1FE4UZJ5V4DOHGG


In [11]:
toronto_data.loc[0, 'Neighborhood']
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=50OG1L24VQHMKEDMSCTEKVCRVM3YZ2LLOOGNCNN4QVKN2SGY&client_secret=QMI5S45Y5GFI0MJT0BGORIW23LEFUNNMU1FE4UZJ5V4DOHGG&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [14]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [15]:
toronto_venues


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
5,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park
6,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
7,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
8,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
9,"Regent Park, Harbourfront",43.65426,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center


In [16]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,63,63,63,63,63,63
Christie,17,17,17,17,17,17
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [17]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 209 uniques categories.


In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
all_restaurants=[]
for col in toronto_onehot.columns: 
    if regex.findall("Restaurant",col):
        all_restaurants.append(col)
all_restaurants

['American Restaurant',
 'Asian Restaurant',
 'Belgian Restaurant',
 'Brazilian Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Colombian Restaurant',
 'Comfort Food Restaurant',
 'Dim Sum Restaurant',
 'Doner Restaurant',
 'Dumpling Restaurant',
 'Eastern European Restaurant',
 'Ethiopian Restaurant',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'Filipino Restaurant',
 'French Restaurant',
 'German Restaurant',
 'Gluten-free Restaurant',
 'Greek Restaurant',
 'Indian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'Modern European Restaurant',
 'Molecular Gastronomy Restaurant',
 'Moroccan Restaurant',
 'New American Restaurant',
 'Persian Restaurant',
 'Portuguese Restaurant',
 'Ramen Restaurant',
 'Restaurant',
 'Seafood Restaurant',
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 'Thai Restaurant',
 'Theme Restaurant',


In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.017857,0.000000,0.000000,0.000000,0.00
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00
2,Central Bay Street,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.015873,0.000000,0.000000,0.015873,0.00
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00
4,Church and Wellesley,0.026667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013333,0.000000,...,0.013333,0.013333,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.00
6,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.010000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.00
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020000,0.000000,0.01,0.00,0.00,0.000000,0.010000,0.010000,0.010000,0.00
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.00
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.043478,0.000000,0.043478,0.014493,0.00


In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 208

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,199th Most Common Venue,200th Most Common Venue,201th Most Common Venue,202th Most Common Venue,203th Most Common Venue,204th Most Common Venue,205th Most Common Venue,206th Most Common Venue,207th Most Common Venue,208th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Cheese Shop,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Restaurant,Farmers Market,...,Middle Eastern Restaurant,Mexican Restaurant,Men's Store,Mediterranean Restaurant,Martial Arts School,Market,Lounge,Lingerie Store,Latin American Restaurant,Yoga Studio
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Rental Car Location,Airport,Airport Food Court,Airport Gate,Harbor / Marina,...,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Men's Store,Mediterranean Restaurant,Martial Arts School,Market,Yoga Studio
2,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Modern European Restaurant,...,Steakhouse,IT Services,Hotel Bar,Strip Club,Hostel,Hospital,Hookah Bar,Hobby Shop,History Museum,Women's Store
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Diner,Restaurant,Italian Restaurant,Candy Store,Baby Store,...,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Men's Store,Mediterranean Restaurant,Martial Arts School,Market,Yoga Studio
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Men's Store,Café,Bubble Tea Shop,...,Italian Restaurant,Lounge,Liquor Store,Lingerie Store,Latin American Restaurant,Lake,Korean Restaurant,Jewelry Store,Jazz Club,Gym / Fitness Center
5,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,...,Irish Pub,Market,Lounge,Liquor Store,Lingerie Store,Lake,Korean Restaurant,Juice Bar,Jewelry Store,Yoga Studio
6,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Asian Restaurant,Steakhouse,Seafood Restaurant,...,Indie Movie Theater,Lingerie Store,Latin American Restaurant,Lake,Korean Restaurant,Juice Bar,Jewelry Store,Jazz Club,Irish Pub,Yoga Studio
7,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Lingerie Store,Italian Restaurant,Hotel,...,Irish Pub,Hobby Shop,Indie Movie Theater,Indian Restaurant,Ice Cream Shop,IT Services,Hotel Bar,Hostel,Hospital,Yoga Studio
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel,Café,Restaurant,Scenic Lookout,Fried Chicken Joint,Brewery,Italian Restaurant,...,Liquor Store,Lingerie Store,Latin American Restaurant,Korean Restaurant,Jewelry Store,Jazz Club,Irish Pub,Hostel,Hospital,Yoga Studio
9,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Vietnamese Restaurant,Bar,Vegetarian / Vegan Restaurant,Mexican Restaurant,Arts & Crafts Store,Park,Pizza Place,...,Miscellaneous Shop,Middle Eastern Restaurant,Men's Store,Mediterranean Restaurant,Martial Arts School,Market,Lounge,Liquor Store,Lingerie Store,Yoga Studio


In [23]:
all_restaurants
asian_restaurants=['Asian Restaurant','Chinese Restaurant','Japanese Restaurant','Korean Restaurant','Sushi Restaurant','Taiwanese Restaurant','Thai Restaurant','Vietnamese Restaurant']

In [24]:
toronto_grouped['asian']=toronto_grouped[asian_restaurants].sum(axis=1)
toronto_grouped['diff']=toronto_grouped['asian']-toronto_grouped['Korean Restaurant']

In [25]:
toronto_grouped[['Neighborhood','diff']]

,Neighborhood,diff
0,Berczy Park,0.053571
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
2,Central Bay Street,0.063492
3,Christie,0.000000
4,Church and Wellesley,0.146667
5,"Commerce Court, Victoria Hotel",0.060000
6,"First Canadian Place, Underground city",0.100000
7,"Garden District, Ryerson",0.070000
8,"Harbourfront East, Union Station, Toronto Islands",0.030000
9,"Kensington Market, Chinatown, Grange Park",0.057971


In [26]:
num=np.argmax(toronto_grouped['asian']-toronto_grouped['Korean Restaurant'])

/Users/lesliezhao/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


In [27]:
toronto_grouped['Neighborhood'][num]

'Church and Wellesley'